# Projeto Preparação de Dados e KNN
### CMC-13 Introdução a Ciência de Dados

### Alunos:
- **João Pedro Couto Vieira**
- **Pedro de Oliveira Ramos**
- **Rafael Duarte Rocha**
- **Vinícius Camelo**

---
## Descrição do Trabalho
O dataset inclui avaliações de livros (0 a 10) feitas por vários usuários. Os dados estão divididos em três arquivos de dados tipo csv (comma separated values) tal como descrito abaixo. Os atributos de cada arquivo são os seguintes:

Arquivo sobre **usuário** ($users.csv$) com 20.702 usuários


* ```user_id``` : identificador do usuário (numérico)

* ```age```,: idade do usuário

* ```city```: Cidade do usuário (identificado por user_id)

* ```state```: Estado do usuário

* ```country```: País do usuário

Arquivo sobre **Livro** ($books.csv$) com 1.024 livros


* ```isbn``` : identificador do livro

* ```book_title``` - Título do livro em inglês,

* ```book_author```: Nome do autor do livro

* ```year_of_publication``` : Ano de publicação do livro

* ```publisher```,: Editora

* ```img_l```: Link para imagem de capa do livro

* ```Language```: Idioma no qual foi escrito o livro

* ```Category```: Tipo de livro, observe que um livro pode pertencer a mais de um tipo (string)

Arquivo de **Avaliação** ($ratings.csv$) com 163.974 avaliações


* ```isbn``` : identificador do livro

* ```user_id``` : identificador do usuário que fez a avaliação (numérico)

* ```rating```: avaliação do livro dado pelo usuário (0 a 10)

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load data
books = pd.read_csv("books.csv", delimiter=";")
users = pd.read_csv("users.csv", delimiter=";")
ratings = pd.read_csv("ratings.csv", delimiter=";")

In [7]:
# Join data in a single dataset
dataset = (
    ratings.join(users.set_index("user_id"), on="user_id")
    .join(books.set_index("isbn"), on="isbn")
    .sort_values(by="user_id")
)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 163974 entries, 0 to 87424
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 163974 non-null  object 
 1   user_id              163974 non-null  int64  
 2   rating               163974 non-null  int64  
 3   age                  20702 non-null   float64
 4   city                 20680 non-null   object 
 5   state                20392 non-null   object 
 6   country              20037 non-null   object 
 7   book_title           163974 non-null  object 
 8   book_author          163974 non-null  object 
 9   year_of_publication  163974 non-null  float64
 10  publisher            163974 non-null  object 
 11  img_l                163974 non-null  object 
 12  Language             163974 non-null  object 
 13  Category             163974 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 18.8+ MB


Após a união dos dataframes, ficamos com as seguintes colunas: 

```isbn```, ```user_id```, ```rating```, ```age```, ```city```, ```state```, ```country```, ```book_title```, ```book_author```, ```year_of_publication```, ```publisher```, ```img_l```, ```Language```, ```Category```


Para nossa análise, é importante mantermos atributos que possam ser comparáveis e descartar informações que são extremamente personalizadas. Para nosso objetivo de classificação via KNN, precisamos manter atributos que sirvam minimamente para a formação de clusters.

Assim, por essa lógica, podemos descartar imediatamente ```user_id``` e ```isbn``` que são identificadores únicos.

Além disso, podemos descartar os atributos de ```city```, ```state```. Estas podem ser úteis para algumas análises cujo tema geográfico é relevante, mas para a nossa análise, não é o caso.

Não estamos trabalhando com capa do livro como um atributo relevante, então podemos descartar ```img_l```.

Percebemos que os atributos de ```age``` e ```country``` apresentam muitas informações faltantes, então optamos por descartá-los.

Nos resta, então, os atributos de ```rating```, ```book_title```, ```book_author```, ```year_of_publication```, ```publisher```, ```Language```, ```Category```.

In [8]:
dataset = dataset.drop(columns=["user_id", "isbn", "city", "state", "img_l", "age", "country"])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 163974 entries, 0 to 87424
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   rating               163974 non-null  int64  
 1   book_title           163974 non-null  object 
 2   book_author          163974 non-null  object 
 3   year_of_publication  163974 non-null  float64
 4   publisher            163974 non-null  object 
 5   Language             163974 non-null  object 
 6   Category             163974 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 10.0+ MB
